In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp.magics
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import display, Markdown, HTML

import parallel_corridors_utils
from shared_utils import styleguide

catalog = intake.open_catalog("./*.yml")
alt.renderers.enable("html")

In [ ]:
# parameters cell
itp_id = 182

In [ ]:
%%capture_parameters
itp_id

# Markdown title {itp_id}

In [ ]:
display(HTML(f"<h1>HTML title {itp_id}</h1>"))

In [ ]:
df = catalog.competitive_route_variability.read()

df = (df[(df.calitp_itp_id == itp_id) & 
         (df.route_group.notna())]
      .reset_index(drop=True)
     )


if itp_id == 182:
    df = df.assign(
        route_id = df.route_id.str.replace('-13153', '').astype(int)
    )

In [ ]:
# Grab parameters for narrative text - clean up formatting here 
operator_name = df.name.iloc[0]
district = df.caltrans_district.iloc[0]
formatted_date = pd.to_datetime(parallel_corridors_utils.SELECTED_DATE).strftime('%m-%d-%Y')

PCT_COMPETITIVE_THRESHOLD = 0.75

stats = parallel_corridors_utils.operator_parallel_competitive_stats(
    itp_id, PCT_COMPETITIVE_THRESHOLD)

pct_parallel = round(stats['parallel_routes'] / stats['num_routes'] * 100,1 )
pct_competitive =  round(stats['competitive_routes'] / stats['num_routes'] * 100,1 )

In [ ]:
display(
    Markdown(
        f"**Bus routes in service**: {stats['num_routes']} "
        "<br>**Parallel routes** to State Highway Network (SHN): "
        f"{stats['parallel_routes']} routes ({pct_parallel}%)"
        f"<br>**Competitive routes** against car travel: {stats['competitive_routes']} routes ({pct_competitive}%)"
        "<br>"
        "Notes: <br>*Parallel routes* are routes where at least 30% of the route falls "
        " within 1 mile of the SHN "
        "and cover at least 10% of the highway's length."
        "<br>"
        "<br>*Competitive routes* are routes where at least "
        f"{int(PCT_COMPETITIVE_THRESHOLD*100)}% of the trips on that route take "
        "no more than 2x the car would. The fastest trip for the bus route is "
        "selected to estimate against the car traveling along the same path "
        "at the same departure hour on the same day. A ratio of bus travel time "
        "to car travel time is taken for all other trips using that car travel time, "
        "and a percentage of all trips that fall within the no-more-than-twice-the-car-travel threshold is calculated for the route."
    )
)

In [ ]:
parallel_corridors_utils.generate_report(df)